<a href="https://colab.research.google.com/github/jonkrohn/DLTFpT/blob/master/notebooks/natural_language_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Preprocessing

In this notebook, we clean up a dataset of natural language data (books from Project Gutenberg) and use word2vec to embed the language in word vectors.

**N.B.:** Some, all or none of these preprocessing steps may be helpful to a given downstream application. 

#### Load dependencies

In [ ]:
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('stopwords')

import string

import gensim
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import Word2Vec

import spacy # for a lemmatization example

from sklearn.manifold import TSNE

import pandas as pd
from bokeh.io import output_notebook, output_file
from bokeh.plotting import show, figure

#### Load Data

In [ ]:
from nltk.corpus import gutenberg

In [ ]:
# a convenient method that handles newlines, as well as tokenizing sentences and words in one shot
gberg_sents = gutenberg.sents()

In [ ]:
gberg_sents[0:6]

In [ ]:
gberg_sents[4][14]

#### Iteratively preprocess a sentence

##### a tokenized sentence: 

In [ ]:
gberg_sents[4]

##### to lowercase: 

In [ ]:
[w.lower() for w in gberg_sents[4]]

##### remove stopwords and punctuation: 

In [ ]:
stpwrds = stopwords.words('english') + list(string.punctuation)

In [ ]:
stpwrds

In [ ]:
[w.lower() for w in gberg_sents[4] if w.lower() not in stpwrds]

##### stem words: 

In [ ]:
stemmer = PorterStemmer()

In [ ]:
[stemmer.stem(w.lower()) for w in gberg_sents[4] if w.lower() not in stpwrds]

##### a lemmatization example: 

In [ ]:
nlp = spacy.load('en_core_web_sm') 

In [ ]:
gutenberg.raw()[291:477]

In [ ]:
spacy_doc = nlp(gutenberg.raw()[291:477])

In [ ]:
[w.lemma_ for w in spacy_doc]

##### handle bigram collocations:

In [ ]:
phrases = Phrases(gberg_sents) # train detector

In [ ]:
bigram = Phraser(phrases) # create a more efficient Phraser object for transforming sentences

In [ ]:
bigram.phrasegrams # output score of each bigram

In [ ]:
tokenized_sentence = "Jon lives in New York City".split()

In [ ]:
tokenized_sentence

In [ ]:
bigram[tokenized_sentence]

#### Preprocess the corpus

In [ ]:
# as in Maas et al. (2001):
# - leave in stop words ("indicative of sentiment")
# - no stemming ("model learns similar representations of words of the same stem when data suggests it")
lower_sents = []
for s in gberg_sents:
    lower_sents.append([w.lower() for w in s if w.lower() not in list(string.punctuation)])

In [ ]:
lower_sents[0:5]

In [ ]:
lower_bigram = Phraser(Phrases(lower_sents))

In [ ]:
lower_bigram.phrasegrams # miss taylor, mr woodhouse, mr weston

In [ ]:
lower_bigram["jon lives in new york city".split()]

In [ ]:
lower_bigram = Phraser(Phrases(lower_sents, min_count=32, threshold=64))
lower_bigram.phrasegrams

In [ ]:
clean_sents = []
for s in lower_sents:
    clean_sents.append(lower_bigram[s])

In [ ]:
clean_sents[0:9]

In [ ]:
clean_sents[6] 

#### Run word2vec

In [ ]:
# max_vocab_size can be used instead of min_count (which has increased here)
# model = Word2Vec(sentences=clean_sents, size=64, 
#                  sg=1, window=10, iter=5,
#                  min_count=10, workers=4)
# model.save('clean_gutenberg_model.w2v')

#### Explore model

In [ ]:
# skip re-training the model with the next line:  
model = gensim.models.Word2Vec.load('clean_gutenberg_model.w2v') 

In [ ]:
len(model.wv.vocab) # would be 17k if we carried out no preprocessing

In [ ]:
model.wv['dog']

In [ ]:
len(model.wv['dog'])

In [ ]:
model.wv.most_similar('dog', topn=3)

In [ ]:
model.wv.most_similar('eat', topn=3)

In [ ]:
model.wv.most_similar('day', topn=3)

In [ ]:
model.wv.most_similar('father', topn=3)

In [ ]:
model.wv.most_similar('ma_am', topn=3) 

In [ ]:
model.wv.doesnt_match("mother father sister brother dog".split())

In [ ]:
model.wv.similarity('father', 'dog')

In [ ]:
model.wv.most_similar(positive=['father', 'woman'], negative=['man']) 

In [ ]:
model.wv.most_similar(positive=['husband', 'woman'], negative=['man']) 

#### Reduce word vector dimensionality with t-SNE

In [ ]:
# tsne = TSNE(n_components=2, n_iter=1000)

In [ ]:
# X_2d = tsne.fit_transform(model.wv[model.wv.vocab])

In [ ]:
# coords_df = pd.DataFrame(X_2d, columns=['x','y'])
# coords_df['token'] = model.wv.vocab.keys()

In [ ]:
# coords_df.head()

In [ ]:
# coords_df.to_csv('clean_gutenberg_tsne.csv', index=False)

#### Visualise 

In [ ]:
coords_df = pd.read_csv('clean_gutenberg_tsne.csv')

In [ ]:
output_notebook()

In [ ]:
subset_df = coords_df.sample(n=5000)

In [ ]:
p = figure(plot_width=800, plot_height=800)
_ = p.text(x=subset_df.x, y=subset_df.y, text=subset_df.token)

In [ ]:
show(p)